In [1]:
pip install langchain

  Using cached aiohttp-3.9.5-cp311-cp311-win_amd64.whl.metadata (7.7 kB)
  Using cached tenacity-8.3.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached orjson-3.10.3-cp311-none-win_amd64.whl.metadata (50 kB)
  Using cached greenlet-3.0.3-cp311-cp311-win_amd64.whl.metadata (3.9 kB)
  Using cached jsonpointer-2.4-py2.py3-none-any.whl.metadata (2.5 kB)
   ---------------------------------------- 0.0/973.5 kB ? eta -:--:--
   ---------------------------------------- 10.2/973.5 kB ? eta -:--:--
   - ------------------------------------- 30.7/973.5 kB 660.6 kB/s eta 0:00:02
   -- ------------------------------------ 61.4/973.5 kB 469.7 kB/s eta 0:00:02
   ---- --------------------------------- 112.6/973.5 kB 726.2 kB/s eta 0:00:02
   ------ ------------------------------- 163.8/973.5 kB 756.6 kB/s eta 0:00:02
   ----------- ---------------------------- 276.5/

In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [4]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [6]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_43dfabdef1', 'finish_reason': 'stop', 'logprobs': None}, id='run-590900f5-bb2d-4be7-bfcf-c411af0b2d76-0')

In [7]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't have access to personal information about users. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 11, 'total_tokens': 32}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_43dfabdef1', 'finish_reason': 'stop', 'logprobs': None}, id='run-239a4323-1c59-42cb-8522-0363bf46d60d-0')

In [8]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='You mentioned that your name is Bob. How can I help you today, Bob?', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 33, 'total_tokens': 50}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_aa87380ac5', 'finish_reason': 'stop', 'logprobs': None}, id='run-34a052b8-2cb0-46a3-843a-95fdfb017ab1-0')

In [10]:
pip install langchain_community

  Using cached dataclasses_json-0.6.6-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.21.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 435.7 kB/s eta 0:00:05
   - -------------------------------------- 0.1/2.1 MB 544.7 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/2.1 MB 722.1 kB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.1 MB 908.0 kB/s eta 0:00:03
   ----- ---------------------------------- 0.3/2.1 MB 1.1 MB/s eta 0:00:02
   --------- ------------------------------ 0.5/2.1 MB 1.6 MB/s eta 0:00:02
   ------------ --------------------------- 0.6/2.1 MB 1.8 MB/s eta 0:00:01
   ------------------ ----------------

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [12]:
config = {"configurable": {"session_id": "abc2"}}

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hi Bob! How can I assist you today?'

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'You mentioned that your name is Bob. How can I help you today, Bob?'

In [15]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, but I don't have access to that information. How can I assist you today?"

In [16]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob. How can I assist you further?'

In [17]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [18]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hi Bob! How can I assist you today?'

In [19]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [20]:
config = {"configurable": {"session_id": "abc5"}}

In [21]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

'Hi Jim! How can I assist you today?'

In [22]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jim. How can I help you today, Jim?'

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [24]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Chinese"}
)

response.content

'嗨，Bob！很高兴见到你。你今天怎么样？'

In [25]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [26]:
config = {"configurable": {"session_id": "abc11"}}

In [27]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Chinese"},
    config=config,
)

response.content

'你好，Todd！很高兴认识你。我能为你做些什么呢？'

In [28]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Chinese"},
    config=config,
)

response.content

'你的名字是Todd。'

In [29]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [30]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [31]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "Chinese",
    }
)
response.content

'抱歉，我无法知道你的名字。你可以告诉我吗？'

In [32]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "Chinese",
    }
)
response.content

'你最喜欢的冰淇淋是香草味的。'

In [33]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [34]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "Chinese",
    },
    config=config,
)

response.content

'对不起，我无法知道你的名字。你可以告诉我吗？'

In [35]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "language": "Chinese",
    },
    config=config,
)

response.content

'对不起，我无法知道你的喜好。你可以告诉我你最喜欢的冰淇淋口味吗？'

In [36]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "Chinese",
    },
    config=config,
):
    print(r.content, end="|")

|嗨|，|Todd|!| |让我|给|你|讲|个|笑|话|吧|：

|有|一天|，小|明|问|妈妈|：“|妈妈|，|为什么|我的|头|发|越来越|少|了|？”
|妈妈|回答|：“|因为|你|每天|都|在|掉|头|发|啊|。”
|小|明|想|了|想|，说|：“|那|我|不|洗|头|，是|不是|头|发|就|不会|掉|了|？”
|妈妈|笑|着|说|：“|不|洗|头|，你|的|朋友|也|会|越来越|少|啊|！”

|希望|你|喜欢|这个|笑|话|！||